In [8]:
#trying out medical embeddings
from gensim.models import KeyedVectors
filename = 'wikipedia-pubmed-and-PMC-w2v.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [14]:
import os
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

osPath = os.path.abspath("Tokenizer.ipynb")
path = 'concept_assertion_relation_training_data/beth/txt/'
files = os.listdir(path)

In [17]:
stop_words = set(stopwords.words('english'))

goods = []
bads = []
for file in files:
    #filter
    if not 'record-' in file:
        continue
    fpath = path+file
    with open(fpath) as iFile:
        for line in iFile:
            words = line.split()
            for w in words: 
                if not w in stop_words: 
                    if w in model:
                        goods.append(w)
                    else:
                        bads.append(w)

In [22]:
len(goods),len(bads),len(goods)/len(bads)

(67593, 2392, 28.25794314381271)